## SunSPoT Testing: 

This file presents an example of using SunSpot model 
The main file is SunSpoT Steps and you can find it on Github: https://github.com/NavidHaghdadi/APVI_SunSPoT


In [1]:
# Import modules for running this notebook
import numpy as np
import pandas as pd
import pickle
import json
import os
import requests
# for displaying the tariffs
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# for visualisation
import plotly # only necessary for plotting
from plotly import tools # only necessary for plotting
import plotly.graph_objs as go # only necessary for plotting
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot # only necessary for plotting
init_notebook_mode(connected=True) # only necessary for plotting
import time

In [2]:
# Importing the saving estimator for testing:
from saving_estimator import saving_est

In [3]:
# 1- user inputs:
user_inputs = json.load(open('user_inputs_Default.json'))
# user doesn't provide data
user_input_load_profile = None
#  OR upload the load profile

# if user provides usage data:
# if has smart meter:
user_inputs['smart_meter']= 'yes'
user_inputs['previous_usage']=[{"total": "N/A", "peak": 300, "offpeak": 300, "shoulder": 300, 
             "start_date": "2018-03-01", "end_date": "2018-05-31"},
  {"total": "N/A", "peak": 300, "offpeak": 400, "shoulder": 400,
             "start_date": "2018-06-01", "end_date": "2018-08-31"}]

# user_inputs['previous_usage']=[{"peak": 300, "offpeak": 300, "shoulder": 300, 
#              "start_date": "2019-08-08", "end_date": "2019-11-21"}]

user_inputs['previous_usage']=[{"total": "N/A", "peak": 300, "offpeak": 300, "shoulder": 300, 
             "start_date": "2018-03-01", "end_date": "2018-05-31"}]
                               
# # if no smart meter
# user_inputs['smart_meter']='no'

# user_inputs['previous_usage']=[{"total": 1500, "peak": "N/A", "offpeak": "N/A", "shoulder": "N/A", 
#              "start_date": "2018-03-01", "end_date": "2018-05-31"},
#   {"total": 1600, "start_date": "2018-06-01", "end_date": "2018-08-31"}]



In [4]:
user_inputs['previous_usage']


[{'total': 'N/A',
  'peak': 300,
  'offpeak': 300,
  'shoulder': 300,
  'start_date': '2018-03-01',
  'end_date': '2018-05-31'}]

In [4]:
# 2- PV profile (generated by other script and the output is hourly in kW:
cwd = os.getcwd()
pv_profile = pd.read_csv(os.path.join(cwd, "PVProfile_Example.csv"))

pv_profile['TS'] = pd.to_datetime(pv_profile['TS'],format='%d/%m/%Y %H:%M')

# 3- PV size. Icelab to provide based on selected roof area
pv_size_kw = 3  # 3 kW

In [5]:
# 4- Tariff:
# Icelab to show a list of tariffs filtered by the location of user based on the distribution_boundaries.geojson file


all_tariffs = requests.get('http://api.ceem.org.au/electricity-tariffs/retail')
all_tariffs = all_tariffs.json()
all_tariffs_list = all_tariffs[0]['Tariffs']
# for i in range(len(all_tariffs)):
#     if all_tariffs[i]['Name'] == Tariff_name:
#         selected_tariff = all_tariffs[i]

# display(selected_tariff)

TariffList=[]
for i in range(len(all_tariffs_list)):
#     display(all_tariffs_list[i]['Name'])
    TariffList.append(all_tariffs_list[i]['Name'])
    
def SelTariff(Tariff):
    for i in range(len(all_tariffs_list)):
        if all_tariffs_list[i]['Name'] == Tariff:
            selected_tariff = all_tariffs_list[i] 
    display(selected_tariff)
    # print(json.dumps(selected_tariff, indent=2))

w=interactive(SelTariff, Tariff=TariffList)
display(w)
# SelTariff('Origin Flat Rate NSW (Endeavour area)')  
           

interactive(children=(Dropdown(description='Tariff', options=('Next Business Energy Block daily Evoenergy area…

In [8]:
# 5- Battery info (zero if no battery). Icelab to obtain from user (with default value of 3kw anf 9kwh)
battery_kw=0
battery_kwh=0

# Distributor (Icelab to provide from the location of customer based on the distribution_boundaries.geojson file)
distributor='Ausgrid'


In [9]:
#  Running the function to get the result (with battery it takes 13 sec in my PC. without battery it's a few seconds)

# assigning the selected tariff
for i in range(len(all_tariffs_list)):
        if all_tariffs_list[i]['Name'] == w.kwargs['Tariff']:
            selected_tariff = all_tariffs_list[i] 
display('Selected tariff is:'+ selected_tariff['Name'])
t0=time.time()
# selected_tariff['Parameters']['Daily']['Value']=0.9

Results=saving_est(user_inputs, pv_profile, selected_tariff,pv_size_kw,battery_kw,battery_kwh,distributor,user_input_load_profile)
print(time.time()-t0)


'Selected tariff is:1st Energy Block daily - Ausgrid area'

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


6.280627489089966


In [10]:
# plotting load profile (for internal checks; not to be included in the website)

LoadProf=pd.read_json(Results['Load_Prof'])
LoadProf[0]=pd.to_datetime(LoadProf[0],unit='ms')
data =[go.Scatter(x= LoadProf[0], y= LoadProf[1],name='Load profile (kWh)', mode='lines')]

layout = go.Layout(title='Load profile' ,xaxis=dict(title='Time'),yaxis=dict(title='Power (kWh)'))
fig = go.Figure(data, layout)
iplot(fig)

LoadProf['Month']=LoadProf[0].dt.month
LoadProf_monthly=LoadProf.groupby('Month').sum()
data =[go.Bar(x= LoadProf_monthly.index, y= LoadProf_monthly[1])]
layout = go.Layout(title='Monthly kWh' ,xaxis=dict(title='Month'),yaxis=dict(title='Energy (kWh)'))
fig = go.Figure(data, layout)
iplot(fig)
display('Annual kWh: ' + LoadProf_monthly[1].sum().astype(int).astype(str) +' kWh')

'Annual kWh: 4128 kWh'

In [11]:
#  Showing the results
# ]f battery doesn't exist:

if battery_kwh == 0:

    display('Estimated annual PV energy generated: '+ Results['Annual_PV_Generation'].astype(int).astype(str) + ' kWh')
    display('Estimated annual PV energy generated per kW installed capacity: '+ Results['Annual_PV_Generation_per_kW'].astype(int).astype(str) + ' kWh')
    display('Estimated PV energy consumed on-site: '+ Results['Est_Annual_PV_self_consumption_SolarOnly'].astype(int).astype(str) + ' kWh')
    display('Estimated annual CO2 emissions avoided: ' + (0.82* Results['Annual_PV_Generation']).astype(int).astype(str)  + ' kg CO2')
    display('Estimated energy exported to grid (No battery): '+ Results['Est_Annual_PV_export_SolarOnly'].astype(int).astype(str) + ' kWh')
    display('Estimated saving due to not using the grid (No battery): '+ Results['Saving_due_to_not_using_grid_SolarOnly'].astype(int).astype(str) + ' $')
    display('Estimated annual Feed-in tariff payment (No battery): '+ Results['FiT_Payment_SolarOnly'].astype(int).astype(str) + ' $')
    display('Estimated total annual saving (No battery): '+ Results['Annual_Saving_SolarOnly'].astype(int).astype(str) + ' $')

else:
    
    display('Estimated annual PV energy generated: '+ Results['Annual_PV_Generation'].astype(int).astype(str) + ' kWh')
    display('Estimated annual PV energy generated per kW installed capacity: '+ Results['Annual_PV_Generation_per_kW'].astype(int).astype(str) + ' kWh')

    display('Estimated annual CO2 emissions avoided: ' + (0.82* Results['Annual_PV_Generation']).astype(int).astype(str)  + ' kg CO2')

    display('Result without battery:')
    display('Estimated PV energy consumed on-site (No Battery): '+ Results['Est_Annual_PV_self_consumption_SolarOnly'].astype(int).astype(str) + ' kWh')
    display('Estimated energy exported to grid (No battery): '+ Results['Est_Annual_PV_export_SolarOnly'].astype(int).astype(str) + ' kWh')
    display('Estimated saving due to not using the grid (No battery): '+ Results['Saving_due_to_not_using_grid_SolarOnly'].astype(int).astype(str) + ' $')
    display('Estimated annual Feed-in tariff payment (No battery): '+ Results['FiT_Payment_SolarOnly'].astype(int).astype(str) + ' $')
    display('Estimated total annual saving (No battery): '+ Results['Annual_Saving_SolarOnly'].astype(int).astype(str) + ' $')

    display('Result with battery:')
    display('Estimated PV energy consumed on-site: '+ Results['Est_Annual_PV_self_consumption_sol_batt'].astype(int).astype(str) + ' kWh')

    display('Estimated energy exported to grid: '+ Results['Est_Annual_PV_export_sol_batt'].astype(int).astype(str) + ' kWh')
    display('Estimated saving due to not using the grid: '+ Results['Saving_due_to_not_using_grid_sol_batt'].astype(int).astype(str) + ' $')
    display('Estimated annual Feed-in tariff payment: '+ Results['FiT_Payment_sol_batt'].astype(int).astype(str) + ' $')
    display('Estimated total annual saving: '+ Results['Annual_Saving_sol_batt'].astype(int).astype(str) + ' $')

# Estimated daily seasonal load vs PV generation

LoadSeasonalPattern=pd.read_json(Results['Load_seasonal_pattern_kW'])
PVSeasonalPattern=pd.read_json(Results['PV_seasonal_pattern_kW'])
if battery_kwh > 0:
    PVBattSeasonalPattern=pd.read_json(Results['pv_batt_seasonal_pattern_kW'])

LoadSeasonalPattern_Sum=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==1,[1,2]]
LoadSeasonalPattern_Fal=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==2,[1,2]]
LoadSeasonalPattern_Win=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==3,[1,2]]
LoadSeasonalPattern_Spr=LoadSeasonalPattern.loc[LoadSeasonalPattern[0]==4,[1,2]]

PVSeasonalPattern_Sum=PVSeasonalPattern.loc[PVSeasonalPattern[0]==1,[1,2]]
PVSeasonalPattern_Fal=PVSeasonalPattern.loc[PVSeasonalPattern[0]==2,[1,2]]
PVSeasonalPattern_Win=PVSeasonalPattern.loc[PVSeasonalPattern[0]==3,[1,2]]
PVSeasonalPattern_Spr=PVSeasonalPattern.loc[PVSeasonalPattern[0]==4,[1,2]]
# if battery exists:

if battery_kwh > 0:
    PVBattSeasonalPattern_Sum=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==1,[1,2]]
    PVBattSeasonalPattern_Fal=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==2,[1,2]]
    PVBattSeasonalPattern_Win=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==3,[1,2]]
    PVBattSeasonalPattern_Spr=PVBattSeasonalPattern.loc[PVBattSeasonalPattern[0]==4,[1,2]]

trace1 = go.Scatter(
    x=LoadSeasonalPattern_Sum[1],
    y=LoadSeasonalPattern_Sum[2],fill='tozeroy', mode= 'none',name='Load (kW) - Summer')
trace11 = go.Scatter(
    x=PVSeasonalPattern_Sum[1],
    y=PVSeasonalPattern_Sum[2],fill='tonexty', mode= 'none',name='PV (kW) - Summer')

if battery_kwh > 0:
    trace12 = go.Scatter(
        x=PVBattSeasonalPattern_Sum[1],
        y=PVBattSeasonalPattern_Sum[2], mode= 'lines',name='Net load (with battery) (kW) - Summer')   
trace13 = go.Scatter(
    x=LoadSeasonalPattern_Sum[1],
    y=LoadSeasonalPattern_Sum[2]-PVSeasonalPattern_Sum[2], mode= 'lines',name='Net load (kW) - Summer')   

trace2 = go.Scatter(
    x=LoadSeasonalPattern_Fal[1],
    y=LoadSeasonalPattern_Fal[2],
    xaxis='x2',yaxis='y2',fill='tozeroy', mode= 'none',name='Load (kW) - Autumn')
trace21 = go.Scatter(
    x=PVSeasonalPattern_Fal[1],
    y=PVSeasonalPattern_Fal[2],
    xaxis='x2',yaxis='y2',fill='tonexty', mode= 'none',name='PV (kW) - Autumn')

if battery_kwh > 0:
    trace22 = go.Scatter(
        x=PVBattSeasonalPattern_Fal[1],
        y=PVBattSeasonalPattern_Fal[2], mode= 'lines',name='Net load (with battery) (kW) - Autumn')  
trace23 = go.Scatter(
    x=LoadSeasonalPattern_Fal[1],
    y=LoadSeasonalPattern_Fal[2]-PVSeasonalPattern_Fal[2], mode= 'lines',name='Net load (kW) - Autumn') 

trace3 = go.Scatter(
    x=LoadSeasonalPattern_Win[1],
    y=LoadSeasonalPattern_Win[2],
    xaxis='x3',yaxis='y3',fill='tozeroy', mode= 'none',name='Load (kW) - Winter')
trace31 = go.Scatter(
    x=PVSeasonalPattern_Win[1],
    y=PVSeasonalPattern_Win[2],
    xaxis='x3',yaxis='y3',fill='tonexty', mode= 'none',name='PV (kW) - Winter')

if battery_kwh > 0:   
    trace32 = go.Scatter(
        x=PVBattSeasonalPattern_Win[1],
        y=PVBattSeasonalPattern_Win[2], mode= 'lines',name='Net load (with battery) (kW) - Winter')  
trace33 = go.Scatter(
    x=LoadSeasonalPattern_Win[1],
    y=LoadSeasonalPattern_Win[2]-PVSeasonalPattern_Win[2], mode= 'lines',name='Net load (kW) - Winter') 

trace4 = go.Scatter(
    x=LoadSeasonalPattern_Spr[1],
    y=LoadSeasonalPattern_Spr[2],
    xaxis='x4',yaxis='y4',fill='tozeroy', mode= 'none',name='Load (kW) - Spring')
trace41 = go.Scatter(
    x=PVSeasonalPattern_Spr[1],
    y=PVSeasonalPattern_Spr[2],
    xaxis='x4',yaxis='y4',fill='tonexty', mode= 'none',name='PV (kW) - Spring')

if battery_kwh > 0:
    trace42 = go.Scatter(
        x=PVBattSeasonalPattern_Spr[1],
        y=PVBattSeasonalPattern_Spr[2], mode= 'lines',name='Net load (with battery) (kW) - Spring') 
trace43 = go.Scatter(
    x=LoadSeasonalPattern_Spr[1],
    y=LoadSeasonalPattern_Spr[2]-PVSeasonalPattern_Spr[2], mode= 'lines',name='Net load (kW) - Spring')  


fig = tools.make_subplots(rows=2, cols=2, print_grid=False, subplot_titles=('Summer', 'Autumn',
                                                          'Winter', 'Spring'))
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace11, 1, 1)
if battery_kwh > 0:
    fig.append_trace(trace12, 1, 1)
fig.append_trace(trace13, 1, 1)

fig.append_trace(trace2, 1, 2)
fig.append_trace(trace21, 1, 2)
if battery_kwh > 0:
    fig.append_trace(trace22, 1, 2)
fig.append_trace(trace23, 1, 2)

fig.append_trace(trace3, 2, 1)
fig.append_trace(trace31, 2, 1)
if battery_kwh > 0:
    fig.append_trace(trace32, 2, 1)
fig.append_trace(trace33, 2, 1)

fig.append_trace(trace4, 2, 2)
fig.append_trace(trace41, 2, 2)
if battery_kwh > 0:
    fig.append_trace(trace42, 2, 2)
fig.append_trace(trace43, 2, 2)

MaxRange=max(max(LoadSeasonalPattern[2]),max(PVSeasonalPattern[2]))

if battery_kwh > 0:
    MinRange=min(min(LoadSeasonalPattern[2]-PVSeasonalPattern[2]),min(PVBattSeasonalPattern[2]))
else:
    MinRange=min(LoadSeasonalPattern[2]-PVSeasonalPattern[2])
    
fig['layout']['yaxis1'].update(title='kW',range=[MinRange,MaxRange])
fig['layout']['yaxis2'].update(title='kW',range=[MinRange,MaxRange])
fig['layout']['yaxis3'].update(title='kW',range=[MinRange,MaxRange])
fig['layout']['yaxis4'].update(title='kW',range=[MinRange,MaxRange])

fig['layout'].update(height=600, width=1000, title='Estimated daily seasonal load vs PV generation',showlegend=True)
# fig = go.Figure(data, layout)
iplot(fig)


        

'Estimated annual PV energy generated: 4062 kWh'

'Estimated annual PV energy generated per kW installed capacity: 1354 kWh'

'Estimated PV energy consumed on-site: 1370 kWh'

'Estimated annual CO2 emissions avoided: 3331 kg CO2'

'Estimated energy exported to grid (No battery): 2691 kWh'

'Estimated saving due to not using the grid (No battery): 393 $'

'Estimated annual Feed-in tariff payment (No battery): 161 $'

'Estimated total annual saving (No battery): 555 $'

In [11]:

fig = { "data": [ { "values": [Results['Annual_PV_Generation']-Results['Est_Annual_PV_export_SolarOnly'],Results['Est_Annual_PV_export_SolarOnly']],
          "labels": ["Self consumption", "Export to the grid"],"domain": {"x": [0, .48]},  "text":["Use", "Export"], "name": "PV Generation (kWh)", 
                       "hoverinfo":"label+percent+value+name",  "hole": .4,  "type": "pie" ,
                   'marker': {'colors': ['rgb(20, 180, 29)',
                                  'rgb(205, 100, 36)']}},  
                     { "values": [Results['New_Bill_SolarOnly'],Results['Saving_due_to_not_using_grid_SolarOnly'], Results['FiT_Payment_SolarOnly']],
          "labels": ["New Bill","Saving due to not using the grid", "Feed in tariff rebate"],
                      "text":["New Bill","Not buying from grid","FiT"],"textposition":"inside","domain": {"x": [.52, 1]},
                      "name": "Annual Bill","hoverinfo":"label+percent+value+name", "hole": .4,"type": "pie",
                   'marker': {'colors': ['rgb(20, 60, 180)',
                                  'rgb(40, 150, 150)',
                                        'rgb(220, 220, 50)']}}],
           "layout": {"title":"Summary of the result (impact of PV)",
                      "annotations": [{ "showarrow": False, "text": 'PV (kWh)',"x": 0.20,"y": 0.5 },
                {  "showarrow": False, "text": "Bill ($)", "x": 0.8, "y": 0.5}] }}
iplot(fig)

if battery_kw > 0:
    fig = { "data": [ { "values": [Results['Annual_PV_Generation']-Results['Est_Annual_PV_export_sol_batt'],Results['Est_Annual_PV_export_sol_batt']],
              "labels": ["Self consumption", "Export to the grid"],"domain": {"x": [0, .48]},  "text":["Use", "Export"], "name": "PV Generation (kWh)", 
                           "hoverinfo":"label+percent+value+name",  "hole": .4,  "type": "pie",
                       'marker': {'colors': ['rgb(20, 180, 29)',
                                      'rgb(205, 100, 36)']}},  
                         { "values": [Results['New_Bill_sol_batt'], Results['Saving_due_to_not_using_grid_sol_batt'], Results['FiT_Payment_sol_batt']],
              "labels": ["New Bill","Saving due to not using the grid", "Feed in tariff rebate"],
                          "text":["New Bill","Not buying from grid","FiT"],"textposition":"inside","domain": {"x": [.52, 1]},
                          "name": "Annual Bill","hoverinfo":"label+percent+value+name", "hole": .4,"type": "pie",
                       'marker': {'colors': ['rgb(20, 60, 180)',
                                      'rgb(40, 150, 150)',
                                            'rgb(220, 220, 50)']}}],
               "layout": {"title":"Summary of the result (impact of PV and Battery)",
                          "annotations": [{ "showarrow": False, "text": 'PV (kWh)',"x": 0.20,"y": 0.5 },
                    {  "showarrow": False, "text": "Bill ($)", "x": 0.8, "y": 0.5}] }}
    iplot(fig)